# Installing Libraries

In [ ]:
# ! pip3 install selenium
# ! pip3 install webdriver_manager

# Importing Libraries

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
import pandas as pd
import time
import os

# Scrape

### Importing the links

In [ ]:
project_links_df = pd.read_excel('project_info_FULL_RUN_11122024_2375.xlsx')
# project_links_df.count()

project_links_df.head()

### Opening up the landing page

In [ ]:
search_url = "https://data.archwaypartnership.uga.edu/"

browser = webdriver.Chrome()

# Open browser to begin search
browser.get(search_url)

# Maximize the window
browser.maximize_window()

time.sleep(2)

### Entering the login credentials

In [ ]:
# entering the username
input_username = browser.find_element(By.ID,'edit-name')
input_username.send_keys("bs83243")

time.sleep(1)

# entering the password
input_password = browser.find_element(By.ID,'edit-pass')
input_password.send_keys("BS@15july1997")


time.sleep(1)

# clicking the login button
button_login = browser.find_element(By.ID,'edit-submit')
button_login.click()
time.sleep(2)

In [ ]:
# # Cancelled Projects

# cancelled_projects_df = project_links_df.query('`Project Status` == "Cancelled" and `Deliverables Exist` == True')
# # cancelled_projects_df.count()
# project_links = cancelled_projects_df["Project Link"]
# failed_downloads = pd.DataFrame(columns=['Project Link', 'Project Link Code', 'Downloaded'])

# project_links.count() 40

In [ ]:
# Completed Projects

completed_projects_df = project_links_df.query('`Project Status` == "Completed" and `Deliverables Exist` == True')
project_links = completed_projects_df["Project Link"]
# project_links.count() 1292
project_links = project_links[100:200]

failed_downloads = pd.DataFrame(columns=['Project Link', 'Project Link Code', 'Downloaded'])

In [ ]:
count = 0

for project_link in project_links:
    count+=1
    
    print(f"We are scraping project number #{count}")
    try:
        browser.get(project_link)
    
        time.sleep(3)
        
        # Click the download link
        download_link = browser.find_element(By.XPATH, '//a[contains(@href, "deliverable-download")]')
        download_link.click()
        
    except Exception as _:

        time.sleep(3)

In [ ]:
#### Check for the failed Downloads

for project_link in project_links:
    project_link_code = project_link.split('/')[-1]
    path = r'C:\Users\bs83243\Downloads\Completed-Nov13'  
    if not os.path.isfile(os.path.join(path, f'{project_link_code}-deliverables.zip')): 
        new_row = pd.DataFrame({'Project Link': [project_link], 'Project Link Code': project_link_code})
        print(f"Failed to download Project with code: {project_link_code}")
        failed_downloads = pd.concat([failed_downloads, new_row], ignore_index=True)

In [ ]:
failed_downloads.head()

In [ ]:
def append_df_to_csv(filename, df):
    # Check if the file exists
    try:
        with open(filename, 'r') as f:
            # File exists, append without header
            df.to_csv(filename, mode='a', header=False, index=False)
    except FileNotFoundError:
        # File does not exist, write with header
        df.to_csv(filename, mode='w', header=True, index=False)

In [ ]:
faild_csv_file = 'failed_downloads.csv'
append_df_to_csv(faild_csv_file, failed_downloads)

In [ ]:
time.sleep(3)
browser.close()
browser.quit()